# Store in PosgreSQL
Test the storage requirements of PostgreSQL by launching a fresh database then evaluating the size of the resultant files

In [1]:
from datetime import datetime, timedelta
from collections import defaultdict
from subprocess import Popen, run
from pathlib import Path
from time import sleep
import pandas as pd
import numpy as np
import psycopg
import shutil
import json
import os

Configuration

In [2]:
to_store = {
    'Current_A': 'NUMERIC(2)',
    'Voltage_V': 'NUMERIC(2)',
    'Cell_Temperature_C': 'NUMERIC(2)',
    'Datetime': 'TIMESTAMP(3) with time zone'
}
rows = 10000

In [3]:
db_path = Path('db')
shutil.rmtree(db_path, ignore_errors=True)

In [4]:
os.environ['PGDATA'] = str(db_path.absolute())

## Prepare PostgreSQL
In a temporary directory that we can study later

In [5]:
run(['pg_ctl', 'initdb'])

The files belonging to this database system will be owned by user "lward".
This user must also own the server process.

The database cluster will be initialized with locale "en_US.UTF-8".
The default database encoding has accordingly been set to "UTF8".
The default text search configuration will be set to "english".

Data page checksums are disabled.

creating directory /home/lward/Work/ROVI/storage-requirement-estimation/evaluate-postgresql/db ... ok
creating subdirectories ... ok
selecting dynamic shared memory implementation ... posix
selecting default max_connections ... 100
selecting default shared_buffers ... 128MB
selecting default time zone ... America/New_York
creating configuration files ... ok
running bootstrap script ... ok
performing post-bootstrap initialization ... ok
syncing data to disk ... ok


Success. You can now start the database server using:

    /home/lward/miniconda3/envs/rovireq/bin/pg_ctl -D /home/lward/Work/ROVI/storage-requirement-estimation/evaluate-postg

initdb: warning: enabling "trust" authentication for local connections
You can change this by editing pg_hba.conf or using the option -A, or
--auth-local and --auth-host, the next time you run initdb.


CompletedProcess(args=['pg_ctl', 'initdb'], returncode=0)

## Load Example Data
Use the XCEL data to for this experiment. We'll need to convert the measurement time column to a `datetime` so it can be treated using PostgreSQL's native time types

In [6]:
xcel = pd.read_csv('../example-data/xcel.csv')
xcel = xcel.query('Cycle_Label != "EIS"').head(rows)
print(f'Loaded {len(xcel)} rows of data')

Loaded 10000 rows of data


In [7]:
xcel['Datetime'] = xcel['Datenum_d'].apply(lambda x: datetime(year=1, month=1, day=1) + timedelta(days=x))

## Save it to Database
Create the table and then add all rows

In [8]:
psql_proc = Popen(['postgres', '-D', db_path.absolute()])
sleep(2)  # For the database to come online

2024-03-25 10:31:59.564 EDT [41452] LOG:  starting PostgreSQL 12.17 on x86_64-conda-linux-gnu, compiled by x86_64-conda-linux-gnu-cc (Anaconda gcc) 11.2.0, 64-bit
2024-03-25 10:31:59.564 EDT [41452] LOG:  listening on IPv4 address "127.0.0.1", port 5432
2024-03-25 10:31:59.565 EDT [41452] LOG:  listening on Unix socket "/tmp/.s.PGSQL.5432"
2024-03-25 10:31:59.575 EDT [41453] LOG:  database system was shut down at 2024-03-25 10:31:59 EDT
2024-03-25 10:31:59.578 EDT [41452] LOG:  database system is ready to accept connections


In [9]:
try:
    with psycopg.connect('postgres://127.0.0.1:5432', dbname='postgres') as conn:
        # Create the table with the required types
        type_str = "\n,\t".join(k + " " + v for k, v in to_store.items())
        conn.execute(f'''CREATE TABLE data (
            {type_str}
        );''')
    
        # Insert data
        with conn.cursor() as cur:
            value_keys = ", ".join(f"%({k})s" for k in to_store.keys())
            cur.executemany(
                f'INSERT INTO data ({", ".join(to_store.keys())}) VALUES ({value_keys});',
                (r.to_dict() for _, r in xcel.iterrows())
            )
finally:
    psql_proc.terminate()
    sleep(2) # For any bits to finish writing

2024-03-25 10:32:02.305 EDT [41452] LOG:  received smart shutdown request
2024-03-25 10:32:02.307 EDT [41452] LOG:  background worker "logical replication launcher" (PID 41459) exited with exit code 1
2024-03-25 10:32:02.307 EDT [41454] LOG:  shutting down
2024-03-25 10:32:02.320 EDT [41452] LOG:  database system is shut down


## Measure Size
Count the size using `du` and report it Bytes

In [10]:
du = run(['du', '--apparent-size', '--block-size=1', '-cs', db_path], capture_output=True, text=True)

In [11]:
total_size = int(du.stdout.split("\n")[0].split()[0])

Save results

In [12]:
with open('storage-results.json', 'a') as fp:
        print(json.dumps({
            'rows': rows,
            'columns': list(to_store.keys()),
            'schema': dict((k, str(v)) for k, v in to_store.items()),
            'size': total_size
        }), file=fp)